In [1]:
%pip install tqdm
%pip install pandas
%pip install geopandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from src.geotif_predictor import GEOTIFPredictor
import src.utils as utils

In [2]:
import os
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import geopandas as gpd

In [5]:


predictor = GEOTIFPredictor()

In [15]:
tif_path = './test_geotif/'
files = [os.path.join(root, file)
         for root, dirs, files in os.walk(tif_path)
         for file in files if file.endswith('66_110.tif') and '.ipynb_checkpoints' not in root]

In [16]:
gdf_d = {}
for file in tqdm(files[:5]):
    gdf_d[file] = predictor.process_image_optimized(file)

  0%|          | 0/1 [00:00<?, ?it/s]


image 1/1 /Users/21010992/Downloads/miem/test_geotif/66_110.tif: 640x640 31 Private sectors, 226.3ms
Speed: 10.3ms preprocess, 226.3ms inference, 32.4ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1/1 [00:17<00:00, 17.28s/it]


In [17]:
combined_data = gpd.GeoDataFrame(pd.concat(gdf_d.values(), ignore_index=True))

In [19]:
combined_data = utils.combined_polygons_gdfs(combined_data)
combined_data = utils.delete_small_objects(combined_data, area_th=10)
smooth_data = utils.apply_rdp_to_gdf(combined_data, tolerance=0.000001)

/Users/21010992/Downloads/miem/src/utils.py:38: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  return combined_gdf.explode().reset_index(drop=True)


In [20]:
combined_data.to_file('./geopackage/66_110.gpkg', driver="GPKG")